# 4 推測統計 確率分布と統計的検定

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

%matplotlib inline

In [12]:
df_id_age = pd.read_excel('978-4-274-22101-9.xlsx', 'ID付きPOSデータ（IDデータ）').iloc[:, 0:3]
df_id_age.head()

,顧客ID,性別,年代
0,1001,2,70
1,1002,2,40
2,1003,2,20
3,1004,2,50
4,1005,2,40


In [13]:
df_id_pos = pd.read_excel('978-4-274-22101-9.xlsx', 'ID付きPOSデータ（POSデータ）')
df_id_pos = pd.merge(df_id_pos, df_id_age, on='顧客ID')
df_id_pos.head()

,レシートNo,日,時間,顧客ID,税抜価格,税抜単価,個数,大カテゴリ番号,大カテゴリ名,中カテゴリ番号,中カテゴリ名,小カテゴリ番号,小カテゴリ名,性別,年代
0,1,1,9,1518,50,10,5,11,農産,1113,野菜,111327,じゃが芋,2,60
1,1,1,9,1518,50,10,5,11,農産,1113,野菜,111363,玉葱,2,60
2,1,1,9,1518,90,90,1,11,農産,1113,野菜,111361,レタス,2,60
3,1,1,9,1518,185,185,1,11,農産,1113,野菜,111339,トマト,2,60
4,438,2,13,1518,265,265,1,13,畜産,1313,精肉加工品,131312,鶏卵,2,60


In [6]:
# 平均購入金額の95%信頼区間
n = df_id_pos['レシートNo'].nunique()
t_dist = stats.t(loc=df_id_pos.groupby(['レシートNo'])['税抜価格'].sum().mean(),
                 scale=np.sqrt(df_id_pos.groupby(['レシートNo'])['税抜価格'].sum().var() / n),
                 df=n-1)
bottom, up = t_dist.interval(alpha=0.95)
print('95% interval: {:.2f} < x < {:.2f}'.format(bottom, up))

95% interval: 1935.96 < x < 2007.70


In [10]:
# パン購買比率95%信頼区間
n = df_id_pos['レシートNo'].nunique()
bread = df_id_pos[df_id_pos['中カテゴリ名'] == 'パン']['レシートNo'].nunique()
bread_rate = bread / n
t_dist = stats.t(loc=bread_rate,
                 scale=np.sqrt(bread_rate * (1 - bread_rate) / n),
                 df=n-1)
bottom, up = t_dist.interval(alpha=0.95)
print('95% interval: {:.3f} < x < {:.3f}'.format(bottom, up))

95% interval: 0.309 < x < 0.331


In [31]:
# 30代と40代の平均購買金額比較、等分散性の検定（上側5％を超えるので下がると判定）

pd.options.display.float_format = '{:.0f}'.format

df_mat = pd.DataFrame(index=['平均', '分散', 'ケース数'])
df_40 = df_id_pos[df_id_pos['年代'] == 40]
df_50 = df_id_pos[df_id_pos['年代'] == 50]
df_40_amount = df_40.groupby(['レシートNo'])['税抜価格'].sum()
df_50_amount = df_50.groupby(['レシートNo'])['税抜価格'].sum()
df_mat['40代'] = [df_40_amount.mean(), df_40_amount.var(), df_40['レシートNo'].nunique()]
df_mat['50代'] = [df_50_amount.mean(), df_50_amount.var(), df_50['レシートNo'].nunique()]
df_mat

,40代,50代
平均,1930,2179
分散,1814064,2735907
ケース数,1643,1261


In [39]:
f = df_mat.loc['分散', '50代'] / df_mat.loc['分散', '40代']
dfx = df_mat.loc['ケース数', '50代'] - 1
dfy = df_mat.loc['ケース数', '40代'] - 1

print('F value:', f)

F value: 1.5081640192740389


In [19]:
# 年代別大カテゴリ紅梅点数集計表から、年代ごとに購買するカテゴリに違いがあるか検定
df_cross_age_bcat = df_id_pos.groupby(['年代', '大カテゴリ名']).size().loc[20:80].unstack()
df_cross_age_bcat

大カテゴリ名,その他,乾物類,加工食品,即席食品,惣菜,水産,畜産,穀物類,菓子,農産,酒類,飲料
年代,,,,,,,,,,,,
20,3,13,172,68,159,40,123,106,157,236,36,185
30,10,33,806,282,622,259,502,441,765,941,170,561
40,7,103,2481,937,2088,831,1632,1710,2035,2537,551,1797
50,11,107,1971,793,2137,823,1222,1073,1566,2142,400,1439
60,17,84,1793,368,1268,785,990,740,907,2443,303,762
70,4,59,1256,262,1049,621,595,340,521,1427,65,469
80,4,22,249,95,217,86,64,110,188,262,26,138


In [22]:
x2, p, dof, expected = stats.chi2_contingency(df_cross_age_bcat)

print("χ square {}".format(x2))
print("probability {}".format(p))  # 有意なので差があると判定
print("degree of feedom {}".format(dof))
print(expected)

χ square 1575.9120249030825
probability 1.204826244161524e-285
degree of feedom 66
[[1.30564736e+00 9.81567035e+00 2.03494468e+02 6.53989438e+01
  1.75796091e+02 8.03206280e+01 1.19559994e+02 1.05384394e+02
  1.43131592e+02 2.32871533e+02 3.61617689e+01 1.24759269e+02]
 [5.42376778e+00 4.07751114e+01 8.45332950e+02 2.71672654e+02
  7.30271591e+02 3.33658572e+02 4.96662164e+02 4.37775542e+02
  5.94580543e+02 9.67367725e+02 1.50218997e+02 5.18260382e+02]
 [1.68074436e+01 1.26355960e+02 2.61956014e+03 8.41872845e+02
  2.26300223e+03 1.03395791e+03 1.53908162e+03 1.35660080e+03
  1.84251600e+03 2.99772762e+03 4.65506161e+02 1.60601126e+03]
 [1.37646214e+01 1.03480457e+02 2.14531456e+03 6.89460052e+02
  1.85330795e+03 8.46770010e+02 1.26044604e+03 1.11100158e+03
  1.50894662e+03 2.45501854e+03 3.81230852e+02 1.31525873e+03]
 [1.05216267e+01 7.91000862e+01 1.63987067e+03 5.27020764e+02
  1.41666188e+03 6.47267926e+02 9.63480385e+02 8.49245581e+02
  1.15343332e+03 1.87660727e+03 2.91411482e+0